In [ ]:
## Captuabsre Picute using face_cascade

import numpy as np
import cv2
from PIL import Image
import pickle

face_cascade=cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt2.xml")
recognizer=cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainner.yml")
labels={}
with open("labels.pickle","rb") as f:
    og_labels=pickle.load(f)
    labels={v:k for k,v in og_labels.items()}


cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=2)
    for (x,y,w,h) in faces:
        roi_gray=gray[y:y+h,x:x+w]
        roi_color=frame[y:y+h,x:x+w]
       
        id_,conf= recognizer.predict(roi_gray)
        if conf >=30:
            font=cv2.FONT_HERSHEY_SIMPLEX
            name=(labels[id_]+str(conf))
            cv2.putText(frame,name,(x,y),font,1,color=(0,255,0),thickness=2)
             
            
        img_item="777.png"
        cv2.imwrite(img_item,roi_color)
        
        #draw a rectraingle
        width=x+w
        height=y+h
        cv2.rectangle(frame,(x,y),(width,height),color=(255,0,0),thickness=6)

    # Display the resulting frame
    cv2.imshow('frame',frame)
    #cv2.imshow('gray',gray)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
import os
from PIL import Image
import pickle

BASE_DIR=os.path.dirname(os.path.abspath("face.py"))
images_dir=os.path.join(BASE_DIR,"images")

face_cascade=cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt2.xml")

recognizer=cv2.face.LBPHFaceRecognizer_create()

current_id=0
label_ids={}
x_train=[]
y_labels=[]

for root,dirs,files in os.walk(images_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path=os.path.join(root,file)
            label=os.path.basename(os.path.dirname(path)).replace(" ","_").lower()
            #print(label,path)
            if not label in label_ids:
                label_ids[label]=current_id
                current_id+=1
            id_=label_ids[label]
            #print(label_ids)
           # y_labels.append(label) # some number for one hot encoding
            #x_train.append(path) # verify this image, turn into NUMPY array, and convert to gray
            
            pil_image=Image.open(path).convert("L") # grayscale
            image_array=np.array(pil_image,"uint8") #normaloze it by dividing it with 255 , we will use it late for CNN
            
            faces=face_cascade.detectMultiScale(image_array,scaleFactor=1.1,minNeighbors=2)
            
            for (x,y,w,h) in faces:
                roi=image_array[y:y+h,x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
                
                
#print (x_train) 
#print (y_labels)


with open("labels.pickle","wb") as f:
    pickle.dump(label_ids,f)
recognizer.train(x_train,np.array(y_labels))
recognizer.save("trainner.yml")

In [4]:
import cv2


In [3]:
import pickle
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
plt.imshow(image_array)